# Imports

In [1]:
import numpy as np
from collections import namedtuple
from functools import reduce
from random import random
from queue import PriorityQueue
from tqdm.auto import tqdm
from math import ceil
from statistics import mean

## Problem Description

In [2]:
State = namedtuple('State', ['taken', 'not_taken'])

PROBLEM_SIZE = 100
NUM_SETS = 200
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )


def goal_check(state):
    return np.all(covered(state))


assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

# A* Search Algorithm

In [3]:
def h(state):
    largest_set_size = max(sum(s) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(covered(state))
    optimistic_estimate = ceil(missing_size / largest_set_size)
    print("largest_set_size: ", largest_set_size)
    print("missing_size: ", missing_size)
    print("optimistic_estimate: ", optimistic_estimate)
    return optimistic_estimate


def h2(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    largest_set_size = max(sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS)
    missing_size = PROBLEM_SIZE - sum(already_covered)
    optimistic_estimate = ceil(missing_size / largest_set_size)
    return optimistic_estimate


def h3(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    # print("taken: ", taken)
    return taken

# possibile miglioramento: scegliere l'utilità basandosi sulla probabilità che un numero appaia sia singolarmente che in coppia con altri numeri
# potrei dare un punteggio basandomi su quanti elementi utili ci sono in ogni set, dove utili è diverso da più numerosi
# ovviamente il primo step prende i più numerosi, ma poi potrebbe essere che un set con meno elementi utili sia più utile di uno con più elementi
# il punteggio potrebbe essere calcolato basandosi sul confronto tra il set e lo stato attuale
def h_nico_che_non_ha_capito_niente(state):
    score = 0
    if np.all(covered(state)):
        return score
    results = []
    already_covered = covered(state)  # array di booleani che indica quali numeri sono già stati coperti e in quale posizione sono
    # for (index, value) in enumerate(already_covered):
    for set in state.not_taken:
        result = sum(np.logical_or(already_covered, set))
        results.append(result)

    max_value = max(results)
    return max_value


# l'obiettivo è dare un punteggio allo state, non al set da prendere, quindi devo valutare un metodo per dare un punteggio basato su quanto sono vicino alla soluzione
# però potrei dare un punteggio allo stato attuale bassandomi sull'analisi dei set che posso scegliere
# potrei analizzare i set candidati e generare per ognuno un valore che indica quanto è utile quel set e poi ritornare il valore piu alto tra tutti i set
def h_nico_che_forse_ha_capito(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing = PROBLEM_SIZE - sum(already_covered)
    utility = []
    # per ogni set che è incluso in state.not_taken conto quanti valori utili ci sono
    # potrei fare +1 per ogni valore utile e poi moltiplicare il risultato per il numero di caselle mancanti
    # candidates = [sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS]
    # for (index, s) in enumerate(state.not_taken):
    temp_score = 0
    # print("State not taken: ", state.not_taken)
    for s in state.not_taken:
        # print("s: ", s)
        for (index, value) in enumerate(SETS[s]):
            if value == True and already_covered[index] == False:
                temp_score += 1  # +1 per ogni valore utile
        utility.append(temp_score * missing)
        temp_score = 0
    
    # toreturn = ceil(mean(utility))
    # toreturn = mean(utility)
    toreturn = max(utility)
    # print("score" , toreturn)
    return toreturn
        

def f(state):
    # return len(state.taken) + h3(state)
    return len(state.taken) + h_nico_che_forse_ha_capito(state)

In [4]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
with tqdm(total=None) as pbar:
    while not goal_check(current_state):
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )
            frontier.put((f(new_state), new_state))
        _, current_state = frontier.get()
        pbar.update(1)

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")
print("Used tiles: ", current_state.taken)

0it [00:00, ?it/s]

Solved in 8 steps (8 tiles)
Used tiles:  {2, 169, 42, 13, 53, 118, 61, 126}


In [5]:
print("Tiles used (hnico): ")
for tile in current_state.taken:
    print(f"Tile {tile} covers {SETS[tile]}")


Tiles used (hnico): 
Tile 2 covers [ True False False False False False False False False False False False
 False False  True False False  True False  True False False False  True
 False False False False False False False False False False False False
  True False  True False False  True False  True False False  True False
 False False  True False False  True False  True False False  True False
 False  True False False False False False False  True  True False False
  True  True  True False False False False False False False False  True
 False False False False False False False  True False False False False
 False False False  True]
Tile 169 covers [ True False  True False  True False  True False  True False False False
 False False False  True False False False False False  True False  True
  True False  True False  True  True  True False  True False  True False
 False  True  True False False False  True False False  True False False
  True False  True False False False False Fals

In [6]:
np.logical_or(np.array([True, True, False, False, False]), np.array([True, False, True, True, False]))

array([ True,  True,  True,  True, False])